In [1]:
import sys
import string 
import re
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from keras.models import load_model
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
MATCH_MULTIPLE_SPACES = re.compile("\ {2,}")
SEQUENCE_LENGTH = 20
vocabulary = open("../data/vocabulary.txt").read().split("\n")
inverse_vocabulary = dict((word, i) for i, word in enumerate(vocabulary))

def words_to_indices(words):
    UNK = '<UNK>'
    return [inverse_vocabulary.get(word, inverse_vocabulary[UNK]) for word in words]

def clean(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, " " + punctuation + " ")
    for i in range(10):
        text = text.replace(str(i), " " + str(i) + " ")
    text = MATCH_MULTIPLE_SPACES.sub(" ", text)
    return text

In [3]:
import random

news = [
    "No one thrown out: UK publishes Brexit plan",
    "No future for Iraqis who worked with US troops",
    "Trump, Tillerson and the South China Sea: What's at stake",
    "Mattis: Only NK need fear missile defense",
    "What went wrong inside the Yemen raid",
    "Israel to build new settlement in West Bank",
    "What you need to know about the Israeli settlements",
    "Palestinians fear UN roadblock despite anti-settlement resolution",
    "Amona evacuation complete amid protests",
    "Cyclist left bloodied after being punched",
    "Police arrest anti-Mugabe pastor",
    "Billionaire snatched from Hong Kong",
    "Romanians protest new corruption law",
    "Proof That Fruitcake Is The Unsung Hero Of Christmas Treats"
]

In [4]:
from ipywidgets import FloatSlider
from ipywidgets import interact, interactive, fixed

In [5]:
import functools
from IPython.display import display

model = load_model('../models/detector.h5')

def predict_news(title):
    return model.predict(sequence.pad_sequences([words_to_indices(clean(title.lower()))], maxlen=SEQUENCE_LENGTH))[0][0]

# Slide Your Tolerance for Bullshit

In [6]:
@interact(tolerance=(0,1,0.01))
def f(tolerance=0.5):
    print('Is This News?\n')
    for i in news:
        print(predict_news(i) > tolerance, '\t', i)

Is This News?

False 	 No one thrown out: UK publishes Brexit plan
False 	 No future for Iraqis who worked with US troops
False 	 Trump, Tillerson and the South China Sea: What's at stake
True 	 Mattis: Only NK need fear missile defense
True 	 What went wrong inside the Yemen raid
True 	 Israel to build new settlement in West Bank
True 	 What you need to know about the Israeli settlements
True 	 Palestinians fear UN roadblock despite anti-settlement resolution
True 	 Amona evacuation complete amid protests
True 	 Cyclist left bloodied after being punched
True 	 Police arrest anti-Mugabe pastor
True 	 Billionaire snatched from Hong Kong
True 	 Romanians protest new corruption law
True 	 Proof That Fruitcake Is The Unsung Hero Of Christmas Treats


In [7]:
@interact(title='Write your own', tolerance=(0, 1, 0.01))
def b(title, tolerance=0.5):
    print('Bullshit' if predict_news(title) > tolerance else 'Real News')

Bullshit
